In [3]:
!pre-commit run --all-files

trim trailing whitespace.................................................Passed
check toml...............................................................Passed
check yaml...............................................................Passed
check json...............................................................Passed
check for merge conflicts................................................Passed
check for added large files..............................................Passed
debug statements (python)................................................Passed
detect private key.......................................................Passed
black....................................................................Passed
flake8...................................................................Passed
shellcheck...........................................(no files to check)Skipped
